In [48]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from fake_useragent import UserAgent
import logging
import time
from random import randint


In [49]:
lien = "https://www.tripadvisor.fr/Search?q=Bikube&geo=187265&ssrc=a&searchNearby=false&searchSessionId=00094ad53a7c5a29.ssid&offset=0"

headers = {
            "User-Agent": UserAgent.random, 
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
            "Accept-Language": "fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7",
            "Connection": "keep-alive",
            "Cache-Control": "no-cache"
        }

In [50]:
content = requests.get(lien, headers={'User-Agent': UserAgent().chrome})
soup = BeautifulSoup(content.text, 'html.parser')


In [51]:
with open('tripadvisor.html', 'w') as f:
    f.write(soup.prettify())

In [47]:
main = soup.find('main')
with open('tripadvisor.html', 'w') as f:
    f.write(main.prettify())
# print(main.prettify())

In [45]:
# data-test-attribute="all-results-section"
# class =SVuzf e
all_results_section = soup.find_all('div', class_='SVuzf e')
print(all_results_section)

[]


In [33]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from fake_useragent import UserAgent
import logging
import time
from random import randint

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def get_soup_from_html(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return BeautifulSoup(file, "html.parser")

def scrape_restaurant_details(soup):
    restaurant_data = {}
    
    # Basic Info
    name = soup.find("h1", class_="biGQs _P hzzSG rRtyp")
    address = soup.find("span", {"data-automation": "restaurantsMapLinkOnName"})
    phone = soup.find("a", href=lambda x: x and x.startswith("tel:"))
    reviews = soup.find("span", class_="biGQs _P pZUbB KxBGd")
    
    restaurant_data["name"] = name.text.strip() if name else "N/A"
    restaurant_data["address"] = address.text.strip() if address else "N/A"
    restaurant_data["phone"] = phone.text.strip() if phone else "N/A"
    restaurant_data["reviews"] = reviews.text.strip() if reviews else "N/A"
    
    # Category and Price
    categories = soup.find_all("span", class_="biGQs _P pZUbB KxBGd")
    if categories and len(categories) >= 2:
        restaurant_data["category"] = categories[0].text.strip()
        restaurant_data["price_range"] = categories[1].text.strip()
    else:
        restaurant_data["category"] = "N/A"
        restaurant_data["price_range"] = "N/A"
    
    # Status
    status = soup.find("div", class_="biGQs _P fOtGX")
    restaurant_data["status"] = status.text.strip() if status else "N/A"
    
    return restaurant_data

def scrape_comments(soup):
    comments_data = []
    comment_containers = soup.find_all("div", class_="review-container")
    
    for container in comment_containers:
        try:
            title = container.find("a", class_="title")
            content = container.find("p", class_="partial_entry")
            rating_tag = container.find("span", class_="ui_bubble_rating")
            date_tag = container.find("span", class_="ratingDate")
            
            comment = {
                "title": title.text.strip() if title else "N/A",
                "content": content.text.strip() if content else "N/A",
                "rating": int(rating_tag["class"][1].split("_")[-1])/10 if rating_tag else None,
                "date": date_tag["title"] if date_tag else "N/A"
            }
            comments_data.append(comment)
            
        except Exception as e:
            logger.error(f"Error extracting comment: {e}")
            
    return comments_data

def main():
    file_path = "test.html"
    soup = get_soup_from_html(file_path)
    
    # Get restaurant details
    restaurant_details = scrape_restaurant_details(soup)
    print("Restaurant Details:")
    for key, value in restaurant_details.items():
        print(f"{key}: {value}")
        
    # Get comments
    comments = scrape_comments(soup)
    print(f"\nFound {len(comments)} comments")
    
    # Export to CSV
    pd.DataFrame([restaurant_details]).to_csv("restaurant_details.csv", 
                                            index=False, 
                                            encoding="utf-8-sig")
    pd.DataFrame(comments).to_csv("restaurant_comments.csv", 
                                 index=False, 
                                 encoding="utf-8-sig")
    
    print("\nData exported to CSV files")

if __name__ == "__main__":
    main()

Restaurant Details:
name: Restaurant Bergamote
address: 123 Rue de Gerland, 69007 Lyon France
phone: +33 4 78 72 64 32
reviews: 159 avis
category: 159 avis
price_range: Nº 30 sur 3 175 restaurants à Lyon
status: Fermé aujourd'hui

Found 0 comments

Data exported to CSV files


In [32]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from fake_useragent import UserAgent
import logging
import time
from random import randint

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def get_headers():
    ua = UserAgent()
    return {
        "User-Agent": ua.random,
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
    }

def make_request(url, retries=3):
    for _ in range(retries):
        try:
            response = requests.get(url, headers=get_headers())
            response.raise_for_status()
            return response.text
        except requests.RequestException as e:
            logger.error(f"Request failed: {str(e)}")
            time.sleep(randint(1, 3))
    return None

def scrape_restaurant_details(soup):
    restaurant_data = {}

    name = soup.find("h1", class_="biGQs _P hzzSG rRtyp")
    restaurant_data["name"] = name.text.strip() if name else "N/A"

    address = soup.find("span", {"data-automation": "restaurantsMapLinkOnName"})
    restaurant_data["address"] = address.text.strip() if address else "N/A"

    phone = soup.find("a", href=lambda x: x and x.startswith("tel:"))
    restaurant_data["phone"] = phone.text.strip() if phone else "N/A"

    reviews = soup.find("span", class_="biGQs _P pZUbB KxBGd")
    restaurant_data["reviews"] = reviews.text.strip() if reviews else "N/A"

    categories = soup.find_all("span", class_="biGQs _P pZUbB KxBGd")
    if categories and len(categories) >= 2:
        restaurant_data["category"] = categories[0].text.strip()
        restaurant_data["price_range"] = categories[1].text.strip()
    else:
        restaurant_data["category"] = "N/A"
        restaurant_data["price_range"] = "N/A"

    status = soup.find("div", class_="biGQs _P fOtGX")
    restaurant_data["status"] = status.text.strip() if status else "N/A"

    return restaurant_data

def scrape_comments(soup):
    comments_data = []

    comment_containers = soup.find_all("div", class_="review-container")

    for container in comment_containers:
        try:
            title = container.find("a", class_="title").text.strip()
            content = container.find("p", class_="partial_entry").text.strip()
            rating_tag = container.find("span", class_="ui_bubble_rating")
            rating = int(rating_tag["class"][1].split("_")[-1]) / 10 if rating_tag else None
            date_tag = container.find("span", class_="ratingDate")
            date = date_tag["title"] if date_tag else "N/A"

            comments_data.append({
                "title": title,
                "content": content,
                "rating": rating,
                "date": date
            })
        except Exception as e:
            logger.error(f"Erreur lors de l'extraction d'un commentaire : {e}")

    return comments_data

def main():
    url = "https://www.tripadvisor.fr/RestaurantSearch?geo=187265&sortOrder=popularity&o=a0"
    content = make_request(url)
    if not content:
        logger.error("Failed to retrieve the page content")
        return

    soup = BeautifulSoup(content, "html.parser")

    restaurant_details = scrape_restaurant_details(soup)
    comments = scrape_comments(soup)

    print("Détails du restaurant :")
    for key, value in restaurant_details.items():
        print(f"{key.capitalize()}: {value}")

    print("\nCommentaires :")
    for comment in comments:
        print(f"- {comment['title']} ({comment['rating']} étoiles) : {comment['content']}")

    details_df = pd.DataFrame([restaurant_details])
    details_df.to_csv("restaurant_details.csv", index=False, encoding="utf-8-sig")

    comments_df = pd.DataFrame(comments)
    comments_df.to_csv("restaurant_comments.csv", index=False, encoding="utf-8-sig")

    print("\nLes détails du restaurant ont été exportés dans 'restaurant_details.csv'.")
    print("Les commentaires ont été exportés dans 'restaurant_comments.csv'.")

if __name__ == "__main__":
    main()

Détails du restaurant :
Name: N/A
Address: N/A
Phone: N/A
Reviews: N/A
Category: N/A
Price_range: N/A
Status: N/A

Commentaires :

Les détails du restaurant ont été exportés dans 'restaurant_details.csv'.
Les commentaires ont été exportés dans 'restaurant_comments.csv'.


In [30]:
from bs4 import BeautifulSoup
import pandas as pd

def scrape_restaurant_details(file_path):
    # Charger le fichier HTML local
    with open(file_path, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")
    
    # Initialiser un dictionnaire pour stocker les données générales
    restaurant_data = {}

    # Récupérer le nom du restaurant
    name = soup.find("h1", class_="biGQs _P hzzSG rRtyp")
    restaurant_data["name"] = name.text.strip() if name else "N/A"

    # Récupérer l'adresse
    address = soup.find("span", {"data-automation": "restaurantsMapLinkOnName"})
    restaurant_data["address"] = address.text.strip() if address else "N/A"

    # Récupérer le numéro de téléphone
    phone = soup.find("a", href=lambda x: x and x.startswith("tel:"))
    restaurant_data["phone"] = phone.text.strip() if phone else "N/A"

    # Récupérer les avis (nombre et note moyenne)
    reviews = soup.find("span", class_="biGQs _P pZUbB KxBGd")
    restaurant_data["reviews"] = reviews.text.strip() if reviews else "N/A"

    # Récupérer la catégorie de cuisine et la gamme de prix
    categories = soup.find_all("span", class_="biGQs _P pZUbB KxBGd")
    if categories and len(categories) >= 2:
        restaurant_data["category"] = categories[0].text.strip()
        restaurant_data["price_range"] = categories[1].text.strip()
    else:
        restaurant_data["category"] = "N/A"
        restaurant_data["price_range"] = "N/A"

    # Récupérer le statut (ouvert ou fermé)
    status = soup.find("div", class_="biGQs _P fOtGX")
    restaurant_data["status"] = status.text.strip() if status else "N/A"

    return restaurant_data

def scrape_comments(file_path):
    # Charger le fichier HTML local
    with open(file_path, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")
    
    # Initialiser une liste pour stocker les commentaires
    comments_data = []

    # Localiser les conteneurs des commentaires
    comment_containers = soup.find_all("div", class_="review-container")

    for container in comment_containers:
        try:
            # Titre du commentaire
            title = container.find("a", class_="title").text.strip()

            # Contenu du commentaire
            content = container.find("p", class_="partial_entry").text.strip()

            # Note (exemple : "bubble_50" -> 5 étoiles)
            rating_tag = container.find("span", class_="ui_bubble_rating")
            rating = int(rating_tag["class"][1].split("_")[-1]) / 10 if rating_tag else None

            # Date de l'avis
            date_tag = container.find("span", class_="ratingDate")
            date = date_tag["title"] if date_tag else "N/A"

            # Ajouter le commentaire au tableau
            comments_data.append({
                "title": title,
                "content": content,
                "rating": rating,
                "date": date
            })
        except Exception as e:
            print(f"Erreur lors de l'extraction d'un commentaire : {e}")

    return comments_data

def main():
    # Spécifiez le chemin vers le fichier HTML
    file_path = "test.html"
    
    # Scraper les détails du restaurant
    restaurant_details = scrape_restaurant_details(file_path)
    
    # Scraper les commentaires
    comments = scrape_comments(file_path)
    
    # Afficher les résultats du restaurant
    print("Détails du restaurant :")
    for key, value in restaurant_details.items():
        print(f"{key.capitalize()}: {value}")
    
    # Afficher les commentaires
    print("\nCommentaires :")
    for comment in comments:
        print(f"- {comment['title']} ({comment['rating']} étoiles) : {comment['content']}")

    # Exporter les résultats dans des fichiers CSV
    details_df = pd.DataFrame([restaurant_details])
    details_df.to_csv("restaurant_details.csv", index=False, encoding="utf-8-sig")

    comments_df = pd.DataFrame(comments)
    comments_df.to_csv("restaurant_comments.csv", index=False, encoding="utf-8-sig")

    print("\nLes détails du restaurant ont été exportés dans 'restaurant_details.csv'.")
    print("Les commentaires ont été exportés dans 'restaurant_comments.csv'.")

if __name__ == "__main__":
    main()


Détails du restaurant :
Name: Restaurant Bergamote
Address: 123 Rue de Gerland, 69007 Lyon France
Phone: +33 4 78 72 64 32
Reviews: 159 avis
Category: 159 avis
Price_range: Nº 30 sur 3 175 restaurants à Lyon
Status: Fermé aujourd'hui

Commentaires :

Les détails du restaurant ont été exportés dans 'restaurant_details.csv'.
Les commentaires ont été exportés dans 'restaurant_comments.csv'.


In [29]:
from bs4 import BeautifulSoup
import pandas as pd


def scrape_restaurant_details(file_path):
    # Charger le fichier HTML local
    with open(file_path, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")
    
    # Initialiser un dictionnaire pour stocker les données
    restaurant_data = {}

    # Récupérer le nom du restaurant
    name = soup.find("h1", class_="biGQs _P hzzSG rRtyp")
    restaurant_data["name"] = name.text.strip() if name else "N/A"

    # Récupérer l'adresse
    address = soup.find("span", {"data-automation": "restaurantsMapLinkOnName"})
    restaurant_data["address"] = address.text.strip() if address else "N/A"

    # Récupérer le numéro de téléphone
    phone = soup.find("a", href=lambda x: x and x.startswith("tel:"))
    restaurant_data["phone"] = phone.text.strip() if phone else "N/A"

    # Récupérer les avis (nombre et note moyenne)
    reviews = soup.find("span", class_="biGQs _P pZUbB KxBGd")
    restaurant_data["reviews"] = reviews.text.strip() if reviews else "N/A"

    # Récupérer la catégorie de cuisine et la gamme de prix
    categories = soup.find_all("span", class_="biGQs _P pZUbB KxBGd")
    if categories and len(categories) >= 2:
        restaurant_data["category"] = categories[0].text.strip()
        restaurant_data["price_range"] = categories[1].text.strip()
    else:
        restaurant_data["category"] = "N/A"
        restaurant_data["price_range"] = "N/A"

    # Récupérer le statut (ouvert ou fermé)
    status = soup.find("div", class_="biGQs _P fOtGX")
    restaurant_data["status"] = status.text.strip() if status else "N/A"

    return restaurant_data


def main():
    # Spécifiez le chemin vers le fichier HTML
    file_path = "test.html"
    
    # Scraper les détails du restaurant
    restaurant_details = scrape_restaurant_details(file_path)
    
    # Afficher les résultats
    for key, value in restaurant_details.items():
        print(f"{key.capitalize()}: {value}")
    
    # Exporter les résultats dans un fichier CSV
    df = pd.DataFrame([restaurant_details])
    df.to_csv("restaurant_details.csv", index=False, encoding="utf-8-sig")
    print("\nLes détails ont été exportés dans 'restaurant_details.csv'.")

if __name__ == "__main__":
    main()


Name: Restaurant Bergamote
Address: 123 Rue de Gerland, 69007 Lyon France
Phone: +33 4 78 72 64 32
Reviews: 159 avis
Category: 159 avis
Price_range: Nº 30 sur 3 175 restaurants à Lyon
Status: Fermé aujourd'hui

Les détails ont été exportés dans 'restaurant_details.csv'.


In [27]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent, FakeUserAgentError
import pandas as pd
import time
from random import randint
import logging
from urllib.parse import urljoin

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class TripAdvisorScraper:
    def __init__(self, base_url):
        """
        Initialise le scraper.
        :param base_url: URL de base pour le scraping
        """
        self.base_url = base_url
        self.setup_user_agent()
        self.session = requests.Session()
        self.min_delay = 2
        self.max_delay = 5

    def setup_user_agent(self):
        """Initialise le User-Agent avec gestion des erreurs."""
        try:
            self.ua = UserAgent()
        except FakeUserAgentError:
            logger.warning("Impossible d'utiliser FakeUserAgent. Utilisation d'un User-Agent par défaut.")
            self.ua = None

    def get_headers(self):
        """Crée des en-têtes HTTP pour simuler un navigateur."""
        default_ua = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        return {
            "User-Agent": self.ua.random if self.ua else default_ua,
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
            "Accept-Language": "fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7",
            "Connection": "keep-alive",
            "Cache-Control": "no-cache"
        }

    def build_url(self, offset):
        """Construit l'URL pour une page donnée."""
        return f"{self.base_url}&o=a{offset}"

    def make_request(self, url, retries=3):
        """Effectue une requête GET avec gestion des erreurs et mise à jour du User-Agent."""
        for attempt in range(retries):
            try:
                headers = self.get_headers()
                response = self.session.get(url, headers=headers, timeout=30)
                response.raise_for_status()
                return response.text
            except Exception as e:
                logger.warning(f"Requête échouée (tentative {attempt + 1}) : {str(e)}")
                if attempt == retries - 1:
                    raise
                time.sleep(randint(3, 7))

    def parse_restaurant(self, element):
        """Extrait les informations d'un restaurant."""
        try:
            name_elem = element.find('a', class_='Lwqic Cj b')
            return {
                'Nom': name_elem.text.strip() if name_elem else "N/A",
                'URL': urljoin(self.base_url, name_elem['href']) if name_elem and name_elem.has_attr('href') else "N/A"
            }
        except Exception as e:
            logger.error(f"Erreur lors de l'analyse d'un restaurant : {str(e)}")
            return None

    def scrape_page(self, offset):
        """Récupère les informations des restaurants sur une page spécifique."""
        url = self.build_url(offset)
        logger.info(f"Scraping des résultats à l'offset {offset} : {url}")

        try:
            content = self.make_request(url)
            soup = BeautifulSoup(content, 'html.parser')

            # Extraction des restaurants
            restaurant_divs = soup.find_all('div', class_='RfBGI')
            restaurants = []

            for div in restaurant_divs:
                data = self.parse_restaurant(div)
                if data:
                    restaurants.append(data)
                time.sleep(randint(1, 2))

            return restaurants
        except Exception as e:
            logger.error(f"Erreur lors du scraping de l'offset {offset} : {str(e)}")
            return []

    def scrape_all_pages(self, max_pages=50):
        """Récupère les informations des restaurants sur plusieurs pages."""
        all_restaurants = []
        empty_pages = 0

        for page_number in range(max_pages):
            offset = page_number * 30
            restaurants = self.scrape_page(offset)
            if restaurants:
                all_restaurants.extend(restaurants)
                empty_pages = 0
                logger.info(f"Offset {offset} : {len(restaurants)} restaurants trouvés.")
            else:
                empty_pages += 1
                if empty_pages >= 2:
                    logger.info("Arrêt du scraping après 2 pages vides consécutives.")
                    break
            time.sleep(randint(self.min_delay, self.max_delay))  # Pause entre les pages
            
        return pd.DataFrame(all_restaurants)

def main():
    base_url = "https://www.tripadvisor.fr/RestaurantSearch?geo=187265&sortOrder=popularity"
    scraper = TripAdvisorScraper(base_url)

    try:
        df = scraper.scrape_all_pages(max_pages=1)  # Scrape jusqu'à 50 pages
        if not df.empty:
            df.to_csv('restaurants_lyon.csv', index=False, encoding='utf-8-sig')
            logger.info(f"Scraping terminé avec succès : {len(df)} restaurants trouvés.")
        else:
            logger.error("Aucun restaurant trouvé.")
    except Exception as e:
        logger.error(f"Échec du scraping : {str(e)}")

if __name__ == "__main__":
    main()

2024-12-22 12:21:46,195 - INFO - Scraping des résultats à l'offset 0 : https://www.tripadvisor.fr/RestaurantSearch?geo=187265&sortOrder=popularity&o=a0
2024-12-22 12:22:39,419 - INFO - Offset 0 : 31 restaurants trouvés.
2024-12-22 12:22:42,426 - INFO - Scraping terminé avec succès : 31 restaurants trouvés.


In [22]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent, FakeUserAgentError
import pandas as pd
import time
from random import randint
import logging

# Configuration du logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class TripAdvisorScraper:
    def __init__(self, base_url, restaurant_name=None):
        """
        Initialise le scraper.
        :param base_url: URL de base pour le scraping
        :param restaurant_name: Nom du restaurant pour filtrer les résultats (facultatif)
        """
        self.base_url = base_url
        self.restaurant_name = restaurant_name
        self.setup_user_agent()
        self.session = requests.Session()
        self.min_delay = 2
        self.max_delay = 5

    def setup_user_agent(self):
        """Initialise le User-Agent avec gestion des erreurs."""
        try:
            self.ua = UserAgent()
        except FakeUserAgentError:
            logger.warning("Impossible d'utiliser FakeUserAgent. Utilisation d'un User-Agent par défaut.")
            self.ua = None

    def get_headers(self):
        """Crée des en-têtes HTTP pour simuler un navigateur."""
        default_ua = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        return {
            "User-Agent": self.ua.random if self.ua else default_ua,
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
            "Accept-Language": "fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7",
            "Connection": "keep-alive",
            "Cache-Control": "no-cache"
        }

    def build_url(self, offset):
        """Construit l'URL pour une page donnée avec un nom de restaurant."""
        if self.restaurant_name:
            # Ajout du nom du restaurant et des paramètres supplémentaires
            query = self.restaurant_name.replace(' ', '+')
            return f"{self.base_url}?q={query}&geo=187265&ssrc=e&searchNearby=false&offset={offset}"
        else:
            raise ValueError("Un nom de restaurant est requis pour cette recherche.")

    def make_request(self, url, retries=3):
        """Effectue une requête GET avec gestion des erreurs."""
        for attempt in range(retries):
            try:
                response = self.session.get(url, headers=self.get_headers(), timeout=30)
                response.raise_for_status()
                return response.text
            except Exception as e:
                logger.warning(f"Requête échouée (tentative {attempt + 1}) : {str(e)}")
                if attempt == retries - 1:
                    raise
                time.sleep(randint(3, 7))

    def scrape_page(self, offset):
        """Récupère les informations des restaurants sur une page spécifique."""
        url = self.build_url(offset)
        logger.info(f"Scraping des résultats à l'offset {offset} : {url}")

        try:
            content = self.make_request(url)
            soup = BeautifulSoup(content, 'html.parser')

            main_div = soup.find('main')
            
            # main_div = BeautifulSoup(main_div, 'html.parser')
            # Extraction des restaurants
            restaurant_divs = main_div.find_all('div', class_='kgrOn o')
           
            restaurants = []

            for div in restaurant_divs:
                a_tag = div.find('a', class_='BMQDV _F Gv wSSLS SwZTJ')
                if a_tag:
                    href = a_tag['href']
                    name = ''.join(a_tag.stripped_strings)
                    # Supprimer les numéros en début de nom, s'il y en a
                    name = name.split(". ", 1)[-1]
                    restaurants.append({'name': name, 'href': href})

            return restaurants
        except Exception as e:
            logger.error(f"Erreur lors du scraping de l'offset {offset} : {str(e)}")
            return []

    def scrape_all_pages(self, max_pages=2):
        """Récupère les informations des restaurants sur plusieurs pages."""
        all_restaurants = []
        empty_pages = 0

        for page_number in range(max_pages):
            offset = page_number * 30
            restaurants = self.scrape_page(offset)
            if restaurants:
                all_restaurants.extend(restaurants)
                empty_pages = 0
                logger.info(f"Offset {offset} : {len(restaurants)} restaurants trouvés.")
            else:
                empty_pages += 1
                if empty_pages >= 2:
                    logger.info("Arrêt du scraping après 2 pages vides consécutives.")
                    break
            time.sleep(randint(self.min_delay, self.max_delay))  # Pause entre les pages
            
        return pd.DataFrame(all_restaurants)

def main():
    base_url = "https://www.tripadvisor.fr/Search"
    restaurant_name = "Bouchon Lyonnais"  # Nom du restaurant pour filtrer
    scraper = TripAdvisorScraper(base_url, restaurant_name)

    try:
        df = scraper.scrape_all_pages(max_pages=10)  # Scrape jusqu'à 10 pages
        if not df.empty:
            df.to_csv('restaurants_lyon.csv', index=False, encoding='utf-8-sig')
            logger.info(f"Scraping terminé avec succès : {len(df)} restaurants trouvés.")
        else:
            logger.error("Aucun restaurant trouvé.")
    except Exception as e:
        logger.error(f"Échec du scraping : {str(e)}")

if __name__ == "__main__":
    main()


2024-12-22 12:04:21,021 - INFO - Scraping des résultats à l'offset 0 : https://www.tripadvisor.fr/Search?q=Bouchon+Lyonnais&geo=187265&ssrc=e&searchNearby=false&offset=0
2024-12-22 12:04:21,364 - ERROR - Erreur lors du scraping de l'offset 0 : 'NoneType' object is not callable
2024-12-22 12:04:24,368 - INFO - Scraping des résultats à l'offset 30 : https://www.tripadvisor.fr/Search?q=Bouchon+Lyonnais&geo=187265&ssrc=e&searchNearby=false&offset=30
2024-12-22 12:04:24,678 - ERROR - Erreur lors du scraping de l'offset 30 : 'NoneType' object is not callable
2024-12-22 12:04:24,678 - INFO - Arrêt du scraping après 2 pages vides consécutives.
2024-12-22 12:04:24,678 - ERROR - Aucun restaurant trouvé.


In [16]:
from bs4 import BeautifulSoup

# Le HTML à analyser
html_content = '''
<div class="RfBGI"><span><a class="Lwqic Cj b" href="/Restaurant_Review-g187265-d28100759-Reviews-Bikube_Lyon_Restaurant-Lyon_Rhone_Auvergne_Rhone_Alpes.html" target="_blank">Bikube Lyon - Restaurant</a></span></div>
<div class="RfBGI"><span><a class="Lwqic Cj b" href="/Restaurant_Review-g187265-d16216238-Reviews-Angelo_Italian_Restaurant-Lyon_Rhone_Auvergne_Rhone_Alpes.html" target="_blank">Angelo | Italian Restaurant</a></span></div>
<div class="RfBGI"><span><a class="Lwqic Cj b" href="/Restaurant_Review-g187265-d15114321-Reviews-L_affreux_Jojo-Lyon_Rhone_Auvergne_Rhone_Alpes.html" target="_blank">1<!-- -->. <!-- -->L'affreux Jojo</a></span></div>
<div class="RfBGI"><span><a class="Lwqic Cj b" href="/Restaurant_Review-g187265-d18626103-Reviews-Table_Partage-Lyon_Rhone_Auvergne_Rhone_Alpes.html" target="_blank">2<!-- -->. <!-- -->Table &amp; Partage</a></span></div>
<div class="RfBGI"><span><a class="Lwqic Cj b" href="/Restaurant_Review-g187265-d23110895-Reviews-Frazarin-Lyon_Rhone_Auvergne_Rhone_Alpes.html" target="_blank">3<!-- -->. <!-- -->Frazarin Bistrot Franco Italien</a></span></div>
<div class="RfBGI"><span><a class="Lwqic Cj b" href="/Restaurant_Review-g187265-d2027277-Reviews-La_Bouteillerie-Lyon_Rhone_Auvergne_Rhone_Alpes.html" target="_blank">4<!-- -->. <!-- -->La Bouteillerie</a></span></div>
<div class="RfBGI"><span><a class="Lwqic Cj b" href="/Restaurant_Review-g187265-d19896976-Reviews-Empanadas_Club-Lyon_Rhone_Auvergne_Rhone_Alpes.html" target="_blank">5<!-- -->. <!-- -->Empanadas Club</a></span></div>
'''

# Création de l'objet BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Extraction des liens et des noms
restaurants = []
for a_tag in soup.find_all('a', class_='Lwqic Cj b'):
    href = a_tag['href']
    # Supprime les éventuels commentaires ou éléments inutiles dans le texte
    name = ''.join(a_tag.stripped_strings)
    restaurants.append({'name': name, 'href': href})

# Affichage des résultats
for restaurant in restaurants:
    print(f"Nom : {restaurant['name']}, Lien : {restaurant['href']}")


Nom : Bikube Lyon - Restaurant, Lien : /Restaurant_Review-g187265-d28100759-Reviews-Bikube_Lyon_Restaurant-Lyon_Rhone_Auvergne_Rhone_Alpes.html
Nom : Angelo | Italian Restaurant, Lien : /Restaurant_Review-g187265-d16216238-Reviews-Angelo_Italian_Restaurant-Lyon_Rhone_Auvergne_Rhone_Alpes.html
Nom : 1.L'affreux Jojo, Lien : /Restaurant_Review-g187265-d15114321-Reviews-L_affreux_Jojo-Lyon_Rhone_Auvergne_Rhone_Alpes.html
Nom : 2.Table & Partage, Lien : /Restaurant_Review-g187265-d18626103-Reviews-Table_Partage-Lyon_Rhone_Auvergne_Rhone_Alpes.html
Nom : 3.Frazarin Bistrot Franco Italien, Lien : /Restaurant_Review-g187265-d23110895-Reviews-Frazarin-Lyon_Rhone_Auvergne_Rhone_Alpes.html
Nom : 4.La Bouteillerie, Lien : /Restaurant_Review-g187265-d2027277-Reviews-La_Bouteillerie-Lyon_Rhone_Auvergne_Rhone_Alpes.html
Nom : 5.Empanadas Club, Lien : /Restaurant_Review-g187265-d19896976-Reviews-Empanadas_Club-Lyon_Rhone_Auvergne_Rhone_Alpes.html
